In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from scipy.stats import norm
import matplotlib.pyplot as plt

In [2]:
def blackScholes_py(S_0, strike, time_to_expiry, implied_vol, riskfree_rate):
    S = S_0
    K = strike
    dt = time_to_expiry
    sigma = implied_vol
    r = riskfree_rate
    Phi = norm.cdf
    d_1 = (np.log(S_0 / K) + (r+sigma**2/2)*dt) / (sigma*np.sqrt(dt))
    d_2 = d_1 - sigma*np.sqrt(dt)
    value =  S*Phi(d_1) - K*np.exp(-r*dt)*Phi(d_2)
    return value

In [3]:
S_0, strike, time_to_expiry, implied_vol, riskfree_rate = 100., 110., 2., 0.2, 0.03
blackScholes_py(S_0, strike, time_to_expiry, implied_vol, riskfree_rate)

9.73983632580859

## B&S Exact Greeks with TensorFlow

In [4]:
@tf.function
def pricer_blackScholes(S0, strike, time_to_expiry, implied_vol, riskfree):
    S       = S0
    K       = strike
    dt      = time_to_expiry
    dt_sqrt = tf.sqrt(dt)
    sigma   = implied_vol
    r       = riskfree
    Phi     = tfp.distributions.Normal(0., 1.).cdf
    
    d1 = (tf.math.log(S / K) + (r + sigma ** 2 / 2) * dt) / (sigma * dt_sqrt)
    d2 = d1 - sigma * dt_sqrt

    npv =  S * Phi(d1) - K * tf.exp(-r * dt) * Phi(d2)
    return npv

In [5]:
S_0 = tf.Variable(100.)
strike = tf.Variable(110.)
time_to_expiry = tf.Variable(2.)
implied_vol = tf.Variable(0.2)
riskfree_rate = tf.Variable(0.03)
pricer_blackScholes(S_0,strike, time_to_expiry, implied_vol, riskfree_rate)

<tf.Tensor: shape=(), dtype=float32, numpy=9.739834>

In [6]:
with tf.GradientTape() as g1:
    npv = pricer_blackScholes(S_0,strike, time_to_expiry, implied_vol, riskfree_rate)
greeks_exact_tf = g1.gradient(npv, [S_0,implied_vol,time_to_expiry,riskfree_rate]) 

## Monte Carlo Greeks with TensorFlow

In [7]:
@tf.function
def brownian(S0, dt, sigma, mu, z):
    dt_sqrt = tf.math.sqrt(dt)
    shock = sigma * dt_sqrt * z
    drift = (mu - (sigma ** 2) / 2)
    bm = tf.math.exp(drift * dt + shock)
    out = S0 * tf.math.cumprod(bm, axis=1)
    return out

@tf.function
def pricer_montecarlo(S0, strike, time_to_expiry, implied_vol, riskfree, z):
    sigma = implied_vol
    T = time_to_expiry
    r = riskfree
    K = strike
    dt = T / z.shape[1]
    st = brownian(S0, dt, sigma, r, z)
    payoff = tf.math.maximum(st[:, -1] - K, 0)
    npv = tf.exp(-r * T) * tf.reduce_mean(payoff)
    return npv

In [8]:
nsims = 10000
nobs = 100
z = tf.random.normal((nsims, nobs), seed=12)
npv = pricer_montecarlo(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, z)
npv

<tf.Tensor: shape=(), dtype=float32, numpy=9.786871>

In [9]:
with tf.GradientTape() as g1:
    npv = pricer_montecarlo(S_0, strike, time_to_expiry, implied_vol, riskfree_rate, z)
greeks_mc_tf = g1.gradient(npv, [S_0, implied_vol,time_to_expiry, riskfree_rate])

In [10]:
greeks_exact_tf

[<tf.Tensor: shape=(), dtype=float32, numpy=0.5066145>,
 <tf.Tensor: shape=(), dtype=float32, numpy=56.411205>,
 <tf.Tensor: shape=(), dtype=float32, numpy=4.0482087>,
 <tf.Tensor: shape=(), dtype=float32, numpy=81.843216>]

In [11]:
greeks_mc_tf

[<tf.Tensor: shape=(), dtype=float32, numpy=0.5079977>,
 <tf.Tensor: shape=(), dtype=float32, numpy=56.693672>,
 <tf.Tensor: shape=(), dtype=float32, numpy=4.065071>,
 <tf.Tensor: shape=(), dtype=float32, numpy=82.0258>]